# 1: Reading Data file.

In [ ]:
import csv
import spacy
import string
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/MyDrive/data.txt','r') as f:
   words = f.read().split()

print(len(words)) #10 seconds.

# 2: Reading Misspellings file.

In [ ]:
misspellings = dict()
with open('/content/drive/MyDrive/misspellings.txt') as csv_file:
  csv_reader = csv.reader(csv_file,delimiter=',')
  index = 0
  for row in csv_reader:
    if index == 0:
      index += 1
      continue
    else:
      temp = row[1].split('\t')
      for i in range(len(temp)):
        temp[i] = temp[i].strip()
      misspellings[row[0]] = temp
      index += 1
misspellings # 3 seconds.

# 3: Generating Unigram Model.




Finds the count of each word, then divides by the total number of words in the data corpus.

In [ ]:
wordOccurrences = dict()
for word in words:
  if word not in wordOccurrences:
    wordOccurrences[word] = 1
  else:
    wordOccurrences[word] += 1

unigrams = dict()
for key, value in wordOccurrences.items():
  unigrams[key] = (value / len(words))


# 4: Creation of insert, delete, substitution and transposition tables.

***4.1 Initializing tables.***

Initializes the tables in form of a dictionary. Alphabets are tuples, which represent keys and values are the occurrences of those alphabets in that order.

In [ ]:
alphabet = string.ascii_lowercase
alphabets = list(alphabet)
alphabets = alphabets + ['#']
insert_table = dict()
delete_table = dict()
substitution_table = dict()
transposition_table = dict()
for u in alphabets:
  for e in alphabets:
    if e == '#':
      continue
    insert_table[u,e] = 0
    delete_table[u,e] = 0
    substitution_table[u,e] = 0
    transposition_table[u,e] = 0
for key,value in insert_table.items():
  print(key,value)

***4.2 Populating tables.***

Iterates over the words in misspellings. For each wrongly spelt word, we find the type of operant used on that particular word i.e. insert, delete, substitution and transpose. After finding the operant used, we incrememt the two alphabets in the respective tables for insert, delete, substitution and transpose.

In [ ]:
def populateTables():
  for correct_word, wrong_words in misspellings.items():
    for wrong_word in wrong_words:
      if len(correct_word) == len(wrong_word): # Either substitution or transpose.
        for i in range(len(correct_word)):
          if correct_word[i] != wrong_word[i]:
            if i != len(wrong_word)-1 and (correct_word[i] == wrong_word[i+1] and correct_word[i+1] == wrong_word[i]): #Transposition.
              u = correct_word[i]
              e = correct_word[i+1]
              transposition_table[u,e] += 1
              break
            else: #Substitution
              u = correct_word[i]
              e = wrong_word[i]
              substitution_table[u,e] += 1
              break
      elif len(wrong_word) > len(correct_word): # Insertion.
        if (correct_word[0] != wrong_word[0]): #Starting letters don't match.
          u = '#'
          e = wrong_word[0]
          insert_table[u,e] += 1
        elif (correct_word[-1] != wrong_word[-1]): #Ending letters don't match.
          u = correct_word[-1]
          e = wrong_word[-1]
          insert_table[u,e] += 1
        else:
          for i in range(len(correct_word)):
            if correct_word[i] != wrong_word[i]: #Middle letters don't match.
              u = correct_word[i-1]
              e = wrong_word[i]
              insert_table[u,e] += 1
              break
          

      elif len(wrong_word) < len(correct_word): #Deletion.
        if (correct_word[0] != wrong_word[0]): #Starting letters don't match.
          u = '#'
          e = correct_word[0]
          delete_table[u,e] += 1
        elif (correct_word[-1] != wrong_word[-1]): #Ending letters don't match.
          u = wrong_word[-1]
          e = correct_word[-1]
          delete_table[u,e] += 1
        else:
          for i in range(len(wrong_word)):
            if correct_word[i] != wrong_word[i]: #Middle letters don't match.
              u = correct_word[i-1]
              e = correct_word[i]
              delete_table[u,e] += 1
              break

populateTables()


# 5: Creating Error Models.

***5.1 Finding character level unigram.***

For each letter in data.txt, we find the occurrences. 

In [ ]:
letterUnigrams = dict()
for word in words:
  for letter in word:
    if letter not in letterUnigrams:
      letterUnigrams[letter] = 1
    else:
      letterUnigrams[letter] += 1
#55 seconds.

***5.2 Finding character level bigrams.***

For each bigram letter in data.txt, we find the occurrences and also add ('#',letter) for our channel model.

In [ ]:
letterBigrams = dict()
for word in words:
  hashBigram = tuple(('#',word[0]))
  if hashBigram not in letterBigrams:
    letterBigrams[hashBigram] = 1
  else:
    letterBigrams[hashBigram] += 1
  for i in range(len(word)-1):
    letter1 = word[i]
    letter2 = word[i+1]
    if tuple((letter1,letter2)) not in letterBigrams:
      letterBigrams[letter1,letter2] = 1
    else:
      letterBigrams[letter1,letter2] += 1
      
letterBigrams # 3 minute 30 seconds.

***5.3 Calculating Edit Distance***

Finds the levenshtein edit distance between two words.

In [ ]:
def calculateEditDistance(word1,word2,size1,size2):
    size1=size1+1
    size2=size2+1

    editDistanceMatrix = {}
    for i in range(size1): 
      editDistanceMatrix[i,0]=i
    for j in range(size2): 
      editDistanceMatrix[0,j]=j
    for i in range(1, size1):
        for j in range(1, size2):
            if word1[i-1] == word2[j-1]:
              cost = 0
            else:
              cost = 1
            editDistanceMatrix[i,j] = min(editDistanceMatrix[i, j-1]+1, 
                                          editDistanceMatrix[i-1, j]+1, 
                                          editDistanceMatrix[i-1, j-1]+cost)
    return editDistanceMatrix[i,j]
print(calculateEditDistance("ussay","usay",5,4))

***5.4 Generating Candidate Words***

From all the unique words in the vocabulary, finds all the candidate words on the basis of edit distance. If edit distance is 1 then it's either one of insertion, deletion or substitution. If it's 2, we check if it's the result of transpose. If it is transpose then we append else we ignore.

In [ ]:
def getCandidateWords(input):
  candidateWordsList = []
  for word in wordOccurrences.keys():
    if len(word) == len(input) or len(word)-1 == len(input) or len(word)+1 == len(input):
      editDist = calculateEditDistance(input,word,len(input),len(word))
      if editDist <= 2:
        if editDist == 1:
          candidateWordsList.append(word)
        elif editDist == 2: #Transpose.
          if len(word) == len(input): # Either substitution or transpose.
            for i in range(len(input)):
              if word[i] != input[i]:
                if i != len(word)-1 and (input[i] == word[i+1] and input[i+1] == word[i]): #Transposition.
                  candidateWordsList.append(word)
                  break
                else:
                  break 
  return candidateWordsList

getCandidateWords("kall")

***5.5 Finding operant used and edited letters.***

Finds the operant used along with the letters involved in that operant and returns these as a list.

In [ ]:
def findEditOperantLetters(inputWord,candidateWord):
  correct_word = candidateWord
  wrong_word = inputWord
  if len(correct_word) == len(wrong_word): # Either substitution or transpose.
    for i in range(len(correct_word)):
      if correct_word[i] != wrong_word[i]:
        if i != len(wrong_word)-1 and (correct_word[i] == wrong_word[i+1] and correct_word[i+1] == wrong_word[i]): #Transposition.
          temp = []
          u = correct_word[i]
          e = correct_word[i+1]
          temp.append(u)
          temp.append(e)
          temp.append("Transpose")
          return temp
        else: #Substitution
          temp = []
          u = correct_word[i]
          e = wrong_word[i]
          temp.append(u)
          temp.append(e)
          temp.append("Substitute")
          return temp
  elif len(wrong_word) > len(correct_word): # Insertion.
    if (correct_word[0] != wrong_word[0]): #Starting letters don't match.
      temp = []
      u = '#'
      e = wrong_word[0]
      temp.append(u)
      temp.append(e)
      temp.append("Insert")
      return temp
    elif (correct_word[-1] != wrong_word[-1]): #Ending letters don't match.
      temp = []
      u = correct_word[-1]
      e = wrong_word[-1]
      temp.append(u)
      temp.append(e)
      temp.append("Insert")
      return temp
    elif (correct_word[-1] == wrong_word[-1]):
      temp = []
      u = correct_word[-1]
      e = wrong_word[-1]
      temp.append(u)
      temp.append(e)
      temp.append("Insert")
      return temp
    else:
      for i in range(len(correct_word)):
        if correct_word[i] != wrong_word[i]: #Middle letters don't match.
          temp = []
          u = correct_word[i-1]
          e = wrong_word[i]
          temp.append(u)
          temp.append(e)
          temp.append("Insert")
          return temp

  elif len(wrong_word) < len(correct_word): #Deletion.
    if (correct_word[0] != wrong_word[0]): #Starting letters don't match.
      temp = []
      u = '#'
      e = correct_word[0]
      temp.append(u)
      temp.append(e)
      temp.append("Delete")
      return temp
    elif (correct_word[-1] != wrong_word[-1]): #Ending letters don't match.
      temp = []
      u = wrong_word[-1]
      e = correct_word[-1]
      temp.append(u)
      temp.append(e)
      temp.append("Delete")
      return temp
    else:
      for i in range(len(wrong_word)):
        if correct_word[i] != wrong_word[i]: #Middle letters don't match.
          temp = []
          u = correct_word[i-1]
          e = correct_word[i]
          temp.append(u)
          temp.append(e)
          temp.append("Delete")
          return temp
#Entered as inputWord, candidateWord. 
findEditOperantLetters("kall","kal")

***5.6 Channel Models and most probable corrected word.*** 

For the input, first we get all the candidate words, then for each of these candidate words we find P(x|w) using the formulas. This is stored in a dictionary. Once we do that, we use this dictionary to find the most probable word for the user's input and display that. If no candidate words were found then it returns a relevant statement.

In [ ]:
def channelModelProbabilities(input):
  probabilities = dict()
  candidateWords = getCandidateWords(input)
  if len(candidateWords) == 0:
    return None
  else:
    for word in candidateWords:
      output = findEditOperantLetters(input,word)
      if output[2] == 'Insert':
        if output[0] == '#':
          probabilities[word] = insert_table[(output[0],output[1])] / len(words)
        else:
          probabilities[word] = insert_table[(output[0],output[1])] / letterUnigrams[output[0]]
      elif output[2] == 'Delete':
        probabilities[word] = delete_table[(output[0],output[1])] / letterBigrams[(output[0],output[1])]
      elif output[2] == 'Transpose':
        probabilities[word] = transposition_table[(output[0],output[1])] / letterBigrams[(output[0],output[1])]
      elif output[2] == 'Substitute':
        probabilities[word] = substitution_table[(output[0],output[1])] / letterUnigrams[output[0]]
    return probabilities

def mostProbableCorrectedWord(input):
  probabilities = channelModelProbabilities(input)
  if probabilities == None:
    return None
  else:
    for key,value in probabilities.items():
      probabilities[key] = value * (unigrams[key]) 
    probableCorrectedWord = max(probabilities,key=probabilities.get)
    return probableCorrectedWord

output = mostProbableCorrectedWord("pak")
if output == None:
  print("No words found...")
else:
  print(output)